# Stick Game

States is a dict to memoize already calculated state using an ID

In [ ]:
states = {}

In [4]:
nbMaxTook = 3
class stateSG():

    def __init__(self, nbStick, maxPlayer):
        self.nbStick = nbStick
        self.maxPlayer = 1 if maxPlayer==True else -1
        
    def calculateScore(self):
        
        if(self.nbStick <= 1):
            return -self.maxPlayer 
        return 0 #To indicate no one win for now. Can be replace by a function which estimate the score
    
    def getChoices(self):
        nbMaxTurn = nbMaxTook if nbMaxTook<= self.nbStick else self.nbStick
        return [i for i in range(1, nbMaxTurn+1)]
        
    def doChoice(self, nb, inNewState = False):
        
        if(inNewState == False):
            self.nbStick -= nb
            self.maxPlayer *= -1
            return None
        else:
            return stateSG(self.nbStick-nb, -self.maxPlayer)
            
    def undoChoice(self, nb, inNewState = False):

        return self.doChoice(-nb, inNewState)

In [5]:
def minmaxDecision(state):
    
    score = state.calculateScore()
    if(score != 0):
        return score
    
    choices = state.getChoices()
    if(state.maxPlayer == 1):
        score = -1
        for choice in choices:
            state.doChoice(choice)
            score = max(score, minmaxDecision(state))
            state.undoChoice(choice)
        return score
    else:
        score = 1
        for choice in choices:
            state.doChoice(choice)
            score = min(score, minmaxDecision(state))
            state.undoChoice(choice)
        return score

In [6]:
for i in range(1, 11):
    test = stateSG(i, True)
    print("Start with ", test.nbStick, " and win" if minmaxDecision(test)==1 else " and lose")

Start with  1  and lose
Start with  2  and win
Start with  3  and win
Start with  4  and win
Start with  5  and lose
Start with  6  and win
Start with  7  and win
Start with  8  and win
Start with  9  and lose
Start with  10  and win


In [9]:
def negamaxDecision(state):
    
    score = state.calculateScore()
    if(score != 0):
        return score * state.maxPlayer
    
    score = -1
    choices = state.getChoices()
    for choice in choices:
        state.doChoice(choice)
        score = max(score,  -negamaxDecision(state))
        state.undoChoice(choice)
    return score

In [10]:
for i in range(1, 11):
    test = stateSG(i, True)
    print("Start with ", test.nbStick, " and win" if negamaxDecision(test)==1 else " and lose")

Start with  1  and lose
Start with  2  and win
Start with  3  and win
Start with  4  and win
Start with  5  and lose
Start with  6  and win
Start with  7  and win
Start with  8  and win
Start with  9  and lose
Start with  10  and win
